In [ ]:
import matplotlib.pyplot as plt
import ipyplot
import numpy as np
import cv2
import math
import statistics
from scipy.signal import find_peaks
from collections import Counter

def imshow(img):
    plt.imshow(img[0:1680, 0:1700])

In [ ]:
img_orig = cv2.imread("jigsawsqr.png")
img_orig = img_orig[70:420, 75:440]
h, w = img_orig.shape[:2]
img_orig = cv2.resize(img_orig, (4*w, 4*h))
imshow(img_orig);

In [ ]:
img_gray = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
imshow(img_gray);

In [ ]:
img_edges = cv2.Canny(image=img_gray, threshold1=100, threshold2=200)
imshow(img_edges);

In [ ]:
_, img_binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
raw_contours, _ = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
img = img_orig.copy()
cv2.drawContours(img, raw_contours, -1, (0, 255, 0), 2)
print("Number of detected contours: ", len(raw_contours))
imshow(img);

In [ ]:
class Item():
    def __init__(self, **kwargs):
        self.update(**kwargs)

    def update(self, **kwargs):
        self.__dict__.update(kwargs)

class LoopingList(list):
    def __getitem__(self, i):
        if isinstance(i, int):
            return super().__getitem__(i % len(self))
        else:
            return super().__getitem__(i)

In [ ]:
areas = [cv2.contourArea(contour) for contour in raw_contours]
median_area = statistics.median(areas)
min_area = 0.5 * median_area
max_area = 2 * median_area
print(f"Ignore too big or small shapes, median_area: {median_area}")

pieces = []
for contour, area in zip(raw_contours, areas):
    if 0.5 < area / median_area < 2:
        rect = cv2.boundingRect(contour)
        piece = Item(contour=contour, area=area, rect=rect)
        pieces.append(piece)

nb_pieces = len(pieces)
print(f"Number of detected pieces: {nb_pieces}")

img = img_orig.copy()
for piece in pieces:
    x, y, w, h = piece.rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

imshow(img);

In [ ]:
img = img_orig.copy()
for piece in pieces:
    min_area_rect = cv2.minAreaRect(piece.contour)
    (cx, cy), (sx, sy), angle = min_area_rect
    if sy < sx:
        angle = (angle + 90) % 360
        sx, sy = sy, sx
    min_area_rect = ((cx, cy), (sx, sy), angle)
    piece.update(min_area_rect=min_area_rect)

    box = np.int0(cv2.boxPoints(min_area_rect))
    cv2.drawContours(img, [box], 0, (0, 255, 0), 2)

imshow(img);

In [ ]:
for piece in pieces:
    x, y, w, h = piece.rect
    img = img_gray[y:y+h, x:x+w]
    piece.update(img=img)

ipyplot.plot_images([piece.img for piece in pieces], img_width=75);

In [ ]:
for piece in pieces:
    piece_degrees = piece.min_area_rect[2]
    x, y, w, h = piece.rect
    img = cv2.linearPolar(img_gray[y:y+h, x:x+w], (w/2, h/2), max(h, w), cv2.WARP_FILL_OUTLIERS)
    y0 = int(piece_degrees * h / 360) % 360
    img = np.concatenate([img[y0:, :], img[:y0, :]])
    _, img_binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    piece.update(img_polar=img_binary)

ipyplot.plot_images([piece.img_polar for piece in pieces], img_width=75); 

In [ ]:
imgs = []

for piece in pieces:
    img_reversed = np.flip(piece.img_polar, axis=1)
    h, w = img_reversed.shape[:2]
    radius = w - np.argmax(img_reversed, axis=1)
    peak_indices = find_peaks(radius, prominence=5)[0]

    polar_peaks = [(radius[idx] * max(h, w) / w, idx * 360 / h) for idx in peak_indices]
    piece.update(polar_peaks=polar_peaks)   # list of [(peak_radius, peak_degrees)]

    img = piece.img_polar.copy()
    for idx in peak_indices:
        cv2.circle(img, (radius[idx], idx), 15, 180, 2)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=150);

In [ ]:
img = img_orig.copy()

for piece in pieces:
    piece_degrees = piece.min_area_rect[2]
    x, y, w, h = piece.rect
    cx = x + w/2
    cy = y + h/2

    for peak_radius, peak_degrees in piece.polar_peaks:
        peak_radians = math.radians(peak_degrees + piece_degrees)
        dx = peak_radius * math.cos(peak_radians)
        dy = peak_radius * math.sin(peak_radians)
        cv2.circle(img, (int(cx + dx), int(cy + dy)), 20, (0, 200, 0), 2)

    peaks = {}  # key=quarter, value=[(abs(angle), angle, distance)] where angles are relative to the piece_angle
    for peak_radius, peak_degrees in piece.polar_peaks:
        quarter = peak_degrees // 90
        diff_degrees = peak_degrees % 180
        if diff_degrees > 90:
            diff_degrees -= 180
        peaks.setdefault(quarter, []).append((abs(diff_degrees), peak_radius, peak_degrees + piece_degrees))

    corners = LoopingList()
    for quarter in range(4):
        _, peak_radius, peak_degrees = min(peaks.get(quarter, [(0, 0, 0)]))
        peak_radians = math.radians(peak_degrees)
        x = cx + peak_radius * math.cos(peak_radians)
        y = cy + peak_radius * math.sin(peak_radians)
        corner = Item(point=(int(x), int(y)))
        corners.append(corner)

    for idx, corner in enumerate(corners):
        corner.update(prev=corners[idx-1], next=corners[idx+1])
    piece.update(corners=corners)
    
    cv2.line(img, piece.corners[0].point, piece.corners[2].point, (255, 0, 0), 3)
    cv2.line(img, piece.corners[1].point, piece.corners[3].point, (255, 0, 0), 3)

imshow(img);

In [ ]:
img = img_orig.copy()

def length(v):
    return v[0]**2 + v[1]**2

def sub_contour(c, idx0, idx1):
    if idx1 > idx0:
        return c[idx0:idx1]
    else:
        return np.concatenate([c[idx0:], c[:idx1]])

for piece in pieces:
    idx_list = LoopingList()
    for corner in piece.corners:
        distances = [length(p[0] - corner.point) for p in piece.contour]
        idx = distances.index(min(distances))
        idx_list.append(idx)
    
    edges = LoopingList()
    for i in range(4):
        contour = sub_contour(piece.contour, idx_list[i], idx_list[i-1])
        edge = Item(
            contour=contour,
            corner0=piece.corners[i],
            corner1=piece.corners[i-1]
        )
        edges.append(edge)
    piece.update(edges=edges)

    cv2.drawContours(img, piece.edges[0].contour, -1, (0, 255, 0), 6)
    cv2.drawContours(img, piece.edges[1].contour, -1, (255, 0, 0), 6)
    cv2.drawContours(img, piece.edges[2].contour, -1, (0, 255, 0), 6)
    cv2.drawContours(img, piece.edges[3].contour, -1, (255, 0, 0), 6)

imshow(img);

In [ ]:
img = img_orig.copy()

for piece in pieces:
    nb_flats = 0
    for idx, edge in enumerate(piece.edges):
        p0 = edge.contour[0][0]
        p1 = edge.contour[-1][0]
        dx, dy = p1 - p0
        angle_radians = math.atan2(dy, dx)
        sin = math.sin(angle_radians)
        cos = math.cos(angle_radians)
        matrix = np.array([[cos, -sin], [sin, cos]])
        normalized_contour = (edge.contour - p0) @ matrix  # first point at (0, 0), last point at (X, 0)
        heights = normalized_contour[:,0,1]
        min_height = min(heights)
        max_height = max(heights)
        if abs(max_height) + abs(min_height) < 10:
            edge_type = 0
            nb_flats += 1
            color = (0, 255, 0)
        elif abs(max_height) > abs(min_height):
            edge_type = 1
            color = (255, 0, 0)
        else:
            edge_type = -1
            color = (0, 0, 255)
        edge.update(
            arc_length=cv2.arcLength(edge.contour, closed=False),
            straight_length=math.sqrt(dx**2 + dy**2),
            height=max(abs(min_height), abs(max_height)),
            angle_degrees=math.degrees(angle_radians),
            type=edge_type,
            prev=piece.edges[idx-1],
            next=piece.edges[idx+1]
        )
        edge.update(
            n_contour=normalized_contour
        )
        cv2.drawContours(img, edge.contour, -1, color, 6)

    piece.update(nb_flats=nb_flats)

imshow(img);

In [ ]:
ipyplot.plot_class_tabs(images=[piece.img for piece in pieces], labels=[piece.nb_flats for piece in pieces], img_width=50)

In [ ]:
nb_flats = Counter([piece.nb_flats for piece in pieces])

assert nb_flats[2] == 4
# H**2 - H*B/2 + I = 0
a = 1
b = - nb_flats[1] / 2
c = nb_flats[0]
delta = b**2 - 4*a*c
inner_height = int((-b - math.sqrt(delta)) / (2*a))
inner_width = int((-b + math.sqrt(delta)) / (2*a))
print(f"Size of puzzle: {2 + inner_width}x{2 + inner_height}")
assert inner_height * inner_width == nb_flats[0]
assert 2 * (inner_height + inner_width) == nb_flats[1]

In [ ]:
all_edges = [edge for piece in pieces for edge in piece.edges]
all_features = [(edge.straight_length, edge.arc_length, edge.height, edge.type, edge.prev.type, edge.next.type) for edge in all_edges]
male_features = np.array([feature for feature in all_features if feature[3] == 1])
female_features = np.array([feature for feature in all_features if feature[3] == -1])

plt.scatter(c=male_features[:, 0], x=male_features[:, 1], y=male_features[:, 2], marker="+")
plt.scatter(c=female_features[:, 0], x=female_features[:, 1], y=female_features[:, 2], marker="x");

In [ ]:
after_flat_male_features = np.array([feature for feature in all_features if feature[3] == 1 and feature[4] == 0])
after_flat_female_features = np.array([feature for feature in all_features if feature[3] == -1 and feature[4] == 0])
before_flat_male_features = np.array([feature for feature in all_features if feature[3] == 1 and feature[5] == 0])
before_flat_female_features = np.array([feature for feature in all_features if feature[3] == -1 and feature[5] == 0])

f = before_flat_male_features; plt.scatter(c=f[:, 0], x=f[:, 1], y=f[:, 2], marker="+")
f = after_flat_female_features; plt.scatter(c=f[:, 0], x=f[:, 1], y=f[:, 2], marker="x")
f = after_flat_male_features; plt.scatter(c=f[:, 0], x=f[:, 1], y=f[:, 2], marker="v")
f = before_flat_female_features; plt.scatter(c=f[:, 0], x=f[:, 1], y=f[:, 2], marker="^");